In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Célula 1 - Instalação das dependências
# @title Instalar requisitos para SoniTranslate
import os
%cd /content
if not os.path.exists('/content/SoniTranslate'):
    !git clone https://github.com/r3gm/SoniTranslate.git

if not os.path.exists('/content/SoniTranslate/whisperX'):
    !cd /content/SoniTranslate && git lfs install && sed -i 's|git+https://github.com/R3gm/whisperX.git@cuda\_11\_8|git+https://github.com/R3gm/whisperX.git@cuda\_12\_x|' requirements\_base.txt


# Instalações condicionais com if not os.path.exists para evitar reinstalações desnecessárias
pacotes_a_instalar = [
    "chex", "pandas-stubs", "ibis-framework", "albumentations", "albucore",
    "pip==23.1.2", "git-lfs",
    "ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/\_packaging/ort-cuda-12-nightly/pypi/simple/"
]

for pacote in pacotes_a_instalar:
    if not os.path.exists(f'/usr/local/lib/python3.10/dist-packages/{pacote.split()[0]}'): # Verifica a existência do pacote principal
      !pip install -q {pacote}

if not os.path.exists('/usr/local/lib/python3.10/dist-packages/piper_tts'):
  Instalar_PIPER_TTS = True # @param {type:"boolean"}
  if Instalar_PIPER_TTS:
    !pip install -q piper-tts==1.2.0

if not os.path.exists('/usr/local/lib/python3.10/dist-packages/TTS'):
  Instalar_Coqui_XTTS = True # @param {type:"boolean"}
  if Instalar_Coqui_XTTS:
    !pip install -q -r /content/SoniTranslate/requirements_xtts.txt
    !pip install -q TTS==0.21.1 --no-deps

# Instala os requirements principais e extras dentro do diretório do projeto
!pip install -q -r /content/SoniTranslate/requirements_base.txt
!pip install -q -r /content/SoniTranslate/requirements_extra.txt

import shutil
import time
from datetime import datetime

def copiar_arquivos_para_drive(output_folder):
    dubbing_folder = "/content/drive/MyDrive/Dubbing"
    os.makedirs(dubbing_folder, exist_ok=True)
    for arquivo in os.listdir(output_folder):
        if arquivo.endswith('.mp4'):
            caminho_origem = os.path.join(output_folder, arquivo)
            caminho_destino = os.path.join(dubbing_folder, arquivo)
            try:
                shutil.copy2(caminho_origem, caminho_destino)
                print(f"Arquivo copiado: {caminho_destino}")
            except Exception as e:
                print(f"Erro ao copiar {arquivo}: {e}")

In [ ]:
# Célula 3 - Executar o SoniTranslate e copiar os arquivos
%cd /content/SoniTranslate
SEU_TOKEN_HF = "" # @param {type:'string'}
%env SEU_TOKEN_HF={SEU_TOKEN_HF}
tema = "Taithrah/Minimal" # @param ["Taithrah/Minimal", "aliabid94/new-theme", "gstaff/xkcd", "ParityError/LimeFace", "abidlabs/pakistan", "rottenlittlecreature/Moon_Goblin", "ysharma/llamas", "gradio/dracula_revamped"]
idioma_interface = "portuguese" # @param ['english', 'portuguese']
nivel_verbosidade = "info" # @param ["debug", "info", "warning", "error", "critical"]

output_folder = "/content/SoniTranslate/outputs"

try:
    !cd /content/SoniTranslate && python app_rvc.py --theme {tema} --verbosity_level {nivel_verbosidade} --language {idioma_interface} --public_url
except KeyboardInterrupt:
    print("Execução interrompida pelo usuário.")
finally:
    print("Copiando arquivos para o Google Drive...")
    copiar_arquivos_para_drive(output_folder)
